In [7]:
%matplotlib inline
from sklearn import tree
from skimage import io
from scipy.stats import kurtosis
from scipy.stats import skew
import cv2
from cv2 import split, threshold
import numpy as np
from matplotlib.pyplot import imshow
import os
WIDTH = 2336
HEIGTH = 3504
NUMBER_OF_PIXELS = WIDTH * HEIGTH
PIXELS_PER_IMAGE = 200000
#wstepne ficzery = wartosc rgb piksela

In [8]:
x_path = '/home/piotr/Desktop/PycharmProjects/Vessels-Recognition/images/others/images/'
y_path = '/home/piotr/Desktop/PycharmProjects/Vessels-Recognition/images/others/manual1/'
test_path = '/home/piotr/Desktop/PycharmProjects/Vessels-Recognition/images/others/manual1/12_dr.JPG'

#test_path = '/home/ns/Documents/Studia/semestr6/iwm/neural-network/data/x/3_3.tif' 

In [24]:
def getFeaturesFromImage(image):
    shape = image.shape
    arr = np.reshape(image, (shape[0]*shape[1], shape[2]))
    return arr

def getClassesFromResult(image):
    shape = image.shape
    _, thresh1 = threshold(image,125,255,cv2.THRESH_BINARY)
    return np.reshape(thresh1, (shape[0]*shape[1]))

def getImageExtract(image, index, maskSize):
    if maskSize % 2 == 0:
        raise ValueError('The size of the mask should be and odd number')
    
    half = maskSize // 2
    
    return image[(index - half):(index + half),(index - half):(index + half)]

def featuresExtractor(extract): #rectangular mask
    #print(extract)
    return [np.mean(extract), np.std(extract), skew(np.ravel(extract)), kurtosis(np.ravel(extract))]

def addFeatures(features, indices, image):
    for i in range(0, len(features)):
        extract = getImageExtract(image, indices[i], 3)
        new_features = featuresExtractor(extract)
        np.append(features[i], new_features)
    
    return features

def train(x_path, y_path, pixels_per_image):
    files = os.listdir(x_path)
    pixels = np.empty([pixels_per_image*len(files), 3]) 
    classes = np.empty([pixels_per_image*len(files)])
    for i, file in enumerate(os.listdir(x_path)):
        indexes = np.random.randint(low=0, high=NUMBER_OF_PIXELS-1, size=pixels_per_image)
        print(str(i+1) + ' of ' + str(len(files)))
        image_x = io.imread(x_path+file)
        features = getFeaturesFromImage(image_x)[indexes]
        features = addFeatures(features, indexes, image_x)
        
        if(i < 1):
            print(len(indexes))
            print(len(features))
        start = i * pixels_per_image
        pixels[start:start+len(features)] = features
        file = file.replace("JPG", "tif")
        file = file.replace("jpg", "tif")
        image_y = io.imread(y_path+file, as_grey=True)
        output = getClassesFromResult(image_y)[indexes]
        classes[start:start+len(output)] = output
    print("Training classifier...")
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(pixels, classes)
    return clf

In [25]:
clf = train(x_path, y_path, PIXELS_PER_IMAGE)
test = io.imread(test_path)
result = clf.predict(getFeaturesFromImage(test))
img_result = np.reshape(result, [WIDTH, HEIGTH])
imshow(img_result)

1 of 45
[[[1 1 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 1 0]]

 [[1 1 0]
  [1 1 0]
  [0 0 0]
  ...
  [0 1 0]
  [0 0 0]
  [0 0 0]]

 [[1 1 0]
  [1 1 0]
  [1 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [1 2 0]
  [1 2 0]
  [1 2 0]]

 [[0 0 0]
  [0 0 0]
  [1 1 1]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]]


/home/piotr/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/piotr/.local/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)


KeyboardInterrupt: 